In [4]:
import pandas as pd

In [5]:
f = pd.read_csv('../Raw_Datasets/items_Giacomo/items.csv',sep=',')
f['TIPO'].unique()

array(['FUENTE', 'INSTALACIÓN DEPORTIVA', 'PISCINA', 'ZONA VERDE',
       'SENDA', 'CENTRO DE SALUD', 'CICLOCARRIL', 'PASO DE BICI',
       'CARRIL BICI', 'VÍA MIXTA', 'SENDA CICLABLE', 'PISTA BICI',
       'ARCEN BICI', 'ACERA BICI', 'ENCAMINAMIENTO', 'CIRCUITO DE OCIO',
       'BANDA RECOMENDADA', 'GIRO PERMITIDO', 'SENTIDO PERMITIDO'],
      dtype=object)

In [11]:
X = pd.read_csv('../Raw_Datasets/items_Giacomo/deportes_final.csv',sep=',')
X['Actividad'].unique()

array(['BMX', 'Ciclismo', 'Ciclismo estacionario', 'Calistenia',
       'Entrenamiento en circuito', 'Levantamiento de peso',
       'Máquina escaladora', 'Aeróbicos', 'Yoga', 'Aeróbicos acuáticos',
       'Correr', 'Atletismo', 'Bádminton', 'Baloncesto', 'Bolos',
       'Billar', 'Boxeo', 'Dardos', 'Frisbee', 'Ultimate frisbee', 'Golf',
       'Minigolf', 'Gimnasia', 'Balonmano', 'Hockey', 'Petanca', 'Calva',
       'Artes marciales', 'Kickboxing', 'Kickball', 'Pádel', 'Raquetbol',
       'Escalada', 'Salto a la comba', 'Chito', 'Skateboard', 'Patinaje',
       'Fútbol', 'Fútbol sala', 'Sóftbol', 'Béisbol',
       'Tenis de mesa / ping pong', 'Tai chi', 'Tenis', 'Tenis en pareja',
       'Voleibol', 'Marcha rápida', 'Montañismo', 'Caminar', 'Natación',
       'Nado sincronizado', 'Waterpolo', 'Voleibol acuático',
       'Patinaje sobre hielo', 'Frontenis', 'Senderismo'], dtype=object)

# Tablas de test

In [7]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder

# Lista de deportes
deportes = np.array(['BMX', 'Ciclismo', 'Ciclismo estacionario', 'Calistenia', 'Entrenamiento en circuito',
                     'Levantamiento de peso', 'Máquina escaladora', 'Aeróbicos', 'Yoga', 'Aeróbicos acuáticos',
                     'Correr', 'Atletismo', 'Bádminton', 'Baloncesto', 'Bolos', 'Billar', 'Boxeo', 'Dardos',
                     'Frisbee', 'Ultimate frisbee', 'Golf', 'Minigolf', 'Gimnasia', 'Balonmano', 'Hockey',
                     'Petanca', 'Calva', 'Artes marciales', 'Kickboxing', 'Kickball', 'Padel', 'Raquetbol',
                     'Escalada', 'Salto a la comba', 'Chito', 'Skateboard', 'Patinaje', 'Fútbol', 'Fútbol sala',
                     'Sóftbol', 'Béisbol', 'Tenis de mesa', 'Tai chi', 'Tenis', 'Tenis en pareja',
                     'Voleibol', 'Marcha rápida', 'Montañismo', 'Caminar', 'Natación', 'Nado sincronizado',
                     'Waterpolo', 'Voleibol acuático', 'Patinaje sobre hielo', 'Frontenis', 'Senderismo'])

# Generar un DataFrame sintético con datos históricos de recomendaciones de deportes
np.random.seed(42)
num_datos = 1000

# Simular datos de Fecha y Hora
# horas = [random.randint(6, 23) for _ in range(num_datos)]

# Simular datos de Clima
clima = np.random.choice(['Soleado',  'Nublado','Lluvioso'], num_datos)

# Simular datos de Temperatura en el rango de 10 a 35 grados Celsius
temperatura = np.random.uniform(10, 40, num_datos)

# Simular datos de Humedad en el rango de 30 a 80 por ciento
humedad = np.random.randint(30, 81, num_datos)

# Simular datos de Edad en el rango de 18 a 65 años
edad = np.random.randint(18, 66, num_datos)

# Simular datos de peso en el rango de 50 a 100 kilogramos
peso = np.random.uniform(50, 100, num_datos)

# Simular objetivos calóricos
objetivos_caloricos = np.random.choice([0, 1, 2], num_datos)

# Simular datos de distancia en el rango de 0.5 a 20 kilómetros
distancia = np.random.uniform(0.5, 20, num_datos)

# Simular condición física
condicion_fisica = np.random.choice([0, 1, 2], num_datos)

# Simular deportes que ha hecho antes

deporte = [random.choice(deportes) for _ in range(1000)]
deporte_practicado = [random.sample(list(deportes), random.randint(3, len(deportes))) for _ in range(num_datos)]

genero = np.random.choice(['Mujer','Hombre'], num_datos)

# Crear el DataFrame
df_historico = pd.DataFrame({
    # 'Hora': horas,
    'Clima': clima,
    'Temperatura (°C)': temperatura.round().astype(int),
    'Humedad': humedad,
    'Edad': edad,
    'Genero': genero,
    'Peso (Kg)': peso.round(2),
    'ObjetivoCalorico': objetivos_caloricos,
    'Distancia (Km)': distancia.round(2),
    'CondicionFisica': condicion_fisica,
    'DeportePracticado': deporte_practicado,
    'Deporte':deporte
})

# Convertir columna de clima en variables dummy
clima_dummies = pd.get_dummies(df_historico['Clima'], prefix='Clima')
df_historico = pd.concat([df_historico, clima_dummies], axis=1)

genero_dummies = pd.get_dummies(df_historico['Genero'], prefix='Genero')
df_historico = pd.concat([df_historico, genero_dummies], axis=1)

# Convertir columna de condición física en variables dummy
condicion_fisica_dummies = pd.get_dummies(df_historico['CondicionFisica'], prefix='Condicion')
df_historico = pd.concat([df_historico, condicion_fisica_dummies], axis=1)

# Convertir columna de objetivo calórico en variables dummy
objetivos_caloricos_dummies = pd.get_dummies(df_historico['ObjetivoCalorico'], prefix='Objetivo')
df_historico = pd.concat([df_historico, objetivos_caloricos_dummies], axis=1)
# label_encoder = LabelEncoder()
# df_historico['Deporte'] = label_encoder.fit_transform(df_historico['Deporte'])


# Convertir columna de deportes practicados en variables dummy
deporte_practicado_dummies = df_historico['DeportePracticado'].apply(lambda x: '|'.join(x)).str.get_dummies()
df_historico = pd.concat([df_historico, deporte_practicado_dummies], axis=1)

# Eliminar columnas originales que ya no son necesarias
df_historico.drop(['Clima', 'CondicionFisica', 'ObjetivoCalorico','Genero','DeportePracticado'], axis=1, inplace=True)

df_historico.to_csv('../Raw_Datasets/datos_historicos_deportes.csv', index=False)
# Mostrar el DataFrame resultan
df_historico


,Temperatura (°C),Humedad,Edad,Peso (Kg),Distancia (Km),Deporte,Clima_Lluvioso,Clima_Nublado,Clima_Soleado,Genero_Hombre,...,Sóftbol,Tai chi,Tenis,Tenis de mesa,Tenis en pareja,Ultimate frisbee,Voleibol,Voleibol acuático,Waterpolo,Yoga
0,12,59,58,50.80,16.16,Calistenia,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,26,43,56,89.11,19.74,Petanca,0,0,1,1,...,1,0,0,0,1,0,0,0,0,0
2,22,35,30,56.22,12.16,Tenis,1,0,0,1,...,1,0,1,1,1,1,1,1,1,1
3,39,32,28,89.88,2.10,Atletismo,1,0,0,1,...,1,1,1,1,1,1,1,1,1,1
4,13,45,63,80.80,13.11,Tenis,0,0,1,0,...,0,1,1,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,38,46,53,78.31,2.93,Bolos,0,1,0,1,...,0,1,1,1,0,0,1,0,0,1
996,13,53,49,50.31,10.75,Voleibol acuático,0,1,0,0,...,1,1,0,1,1,1,1,0,1,0
997,35,50,49,53.09,2.64,Golf,1,0,0,0,...,1,1,1,0,1,1,1,1,0,1
998,21,80,21,76.77,4.46,Boxeo,1,0,0,1,...,1,1,1,0,1,1,1,1,1,0


# Tabla de el EDA

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder

# Lista de deportes
deportes = np.array(['BMX', 'Ciclismo', 'Ciclismo estacionario', 'Calistenia', 'Entrenamiento en circuito',
                     'Levantamiento de peso', 'Máquina escaladora', 'Aeróbicos', 'Yoga', 'Aeróbicos acuáticos',
                     'Correr', 'Atletismo', 'Bádminton', 'Baloncesto', 'Bolos', 'Billar', 'Boxeo', 'Dardos',
                     'Frisbee', 'Ultimate frisbee', 'Golf', 'Minigolf', 'Gimnasia', 'Balonmano', 'Hockey',
                     'Petanca', 'Calva', 'Artes marciales', 'Kickboxing', 'Kickball', 'Padel', 'Raquetbol',
                     'Escalada', 'Salto a la comba', 'Chito', 'Skateboard', 'Patinaje', 'Fútbol', 'Fútbol sala',
                     'Sóftbol', 'Béisbol', 'Tenis de mesa', 'Tai chi', 'Tenis', 'Tenis en pareja',
                     'Voleibol', 'Marcha rápida', 'Montañismo', 'Caminar', 'Natación', 'Nado sincronizado',
                     'Waterpolo', 'Voleibol acuático', 'Patinaje sobre hielo', 'Frontenis', 'Senderismo'])

# Generar un DataFrame sintético con datos históricos de recomendaciones de deportes
np.random.seed(42)
num_datos = 1000

# Simular datos de Fecha y Hora
# horas = [random.randint(6, 23) for _ in range(num_datos)]

# Simular datos de Clima
clima = np.random.choice(['Soleado',  'Nublado','Lluvioso'], num_datos)

# Simular datos de Temperatura en el rango de 10 a 35 grados Celsius
temperatura = np.random.uniform(10, 40, num_datos)

# Simular datos de Humedad en el rango de 30 a 80 por ciento
humedad = np.random.randint(30, 81, num_datos)

# Simular datos de Edad en el rango de 18 a 65 años
edad = np.random.randint(18, 66, num_datos)

# Simular datos de peso en el rango de 50 a 100 kilogramos
peso = np.random.uniform(50, 100, num_datos)

# Simular objetivos calóricos
objetivos_caloricos = np.random.choice([0, 1, 2], num_datos)

# Simular datos de distancia en el rango de 0.5 a 20 kilómetros
distancia = np.random.uniform(0.5, 20, num_datos)

# Simular condición física
condicion_fisica = np.random.choice([0, 1, 2], num_datos)

# Simular deportes que ha hecho antes
deporte = [random.choice(deportes) for _ in range(1000)]
deporte_practicado = [random.sample(list(deportes), random.randint(3, len(deportes))) for _ in range(num_datos)]
prob_hombre = 0.7
prob_mujer = 1 - prob_hombre
genero = np.random.choice(['Hombre', 'Mujer'], num_datos, p=[prob_hombre, prob_mujer])
# genero = np.random.choice(['Mujer','Hombre'], num_datos)

# Crear el DataFrame
df_historico = pd.DataFrame({
    # 'Hora': horas,
    'Clima': clima,
    'Temperatura (°C)': temperatura.round().astype(int),
    'Humedad': humedad,
    'Edad': edad,
    'Genero': genero,
    'Peso (Kg)': peso.round(2),
    'ObjetivoCalorico': objetivos_caloricos,
    'Distancia (Km)': distancia.round(2),
    'CondicionFisica': condicion_fisica,
    'DeportePracticado': deporte_practicado,
    
})

# # Convertir columna de clima en variables dummy
# clima_dummies = pd.get_dummies(df_historico['Clima'], prefix='Clima')
# df_historico = pd.concat([df_historico, clima_dummies], axis=1)

# genero_dummies = pd.get_dummies(df_historico['Genero'], prefix='Genero')
# df_historico = pd.concat([df_historico, genero_dummies], axis=1)

# # Convertir columna de condición física en variables dummy
# condicion_fisica_dummies = pd.get_dummies(df_historico['CondicionFisica'], prefix='Condicion')
# df_historico = pd.concat([df_historico, condicion_fisica_dummies], axis=1)

# # Convertir columna de objetivo calórico en variables dummy
# objetivos_caloricos_dummies = pd.get_dummies(df_historico['ObjetivoCalorico'], prefix='Objetivo')
# df_historico = pd.concat([df_historico, objetivos_caloricos_dummies], axis=1)



# # Convertir columna de deportes practicados en variables dummy
# deporte_practicado_dummies = df_historico['DeportePracticado'].apply(lambda x: '|'.join(x)).str.get_dummies()
# df_historico = pd.concat([df_historico, deporte_practicado_dummies], axis=1)

# Eliminar columnas originales que ya no son necesarias
# df_historico.drop(['Clima', 'CondicionFisica', 'ObjetivoCalorico','Genero','DeportePracticado'], axis=1, inplace=True)

df_historico.to_csv('../Raw_Datasets/df_historicos_deportes.csv', index=False)
# Mostrar el DataFrame resultan
df_historico

,Temperatura (°C),Humedad,Edad,Peso (Kg),Distancia (Km),Clima_Lluvioso,Clima_Nublado,Clima_Soleado,Genero_Hombre,Genero_Mujer,...,Sóftbol,Tai chi,Tenis,Tenis de mesa,Tenis en pareja,Ultimate frisbee,Voleibol,Voleibol acuático,Waterpolo,Yoga
0,12,59,58,50.80,16.16,1,0,0,0,1,...,0,0,0,0,1,1,0,0,0,0
1,26,43,56,89.11,19.74,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,0
2,22,35,30,56.22,12.16,1,0,0,0,1,...,1,1,1,1,1,1,1,1,1,1
3,39,32,28,89.88,2.10,1,0,0,1,0,...,1,1,0,1,0,1,0,0,0,0
4,13,45,63,80.80,13.11,0,0,1,1,0,...,1,1,0,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,38,46,53,78.31,2.93,0,1,0,1,0,...,1,1,1,1,1,0,1,1,1,1
996,13,53,49,50.31,10.75,0,1,0,1,0,...,1,1,1,1,1,1,1,1,1,0
997,35,50,49,53.09,2.64,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
998,21,80,21,76.77,4.46,1,0,0,1,0,...,1,0,1,1,1,0,1,1,1,0


In [3]:
df_historico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 72 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Temperatura (°C)           1000 non-null   int32  
 1   Humedad                    1000 non-null   int32  
 2   Edad                       1000 non-null   int32  
 3   Peso (Kg)                  1000 non-null   float64
 4   Distancia (Km)             1000 non-null   float64
 5   Clima_Lluvioso             1000 non-null   uint8  
 6   Clima_Nublado              1000 non-null   uint8  
 7   Clima_Soleado              1000 non-null   uint8  
 8   Genero_Hombre              1000 non-null   uint8  
 9   Genero_Mujer               1000 non-null   uint8  
 10  Condicion_0                1000 non-null   uint8  
 11  Condicion_1                1000 non-null   uint8  
 12  Condicion_2                1000 non-null   uint8  
 13  Objetivo_0                 1000 non-null   uint8 

# ESTOY TRABAJANDO AQUI 

In [2]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder

# Lista de deportes
deportes = np.array(['BMX', 'Ciclismo', 'Ciclismo estacionario', 'Calistenia', 'Entrenamiento en circuito',
                     'Levantamiento de peso', 'Máquina escaladora', 'Aeróbicos', 'Yoga', 'Aeróbicos acuáticos',
                     'Correr', 'Atletismo', 'Bádminton', 'Baloncesto', 'Bolos', 'Billar', 'Boxeo', 'Dardos',
                     'Frisbee', 'Ultimate frisbee', 'Golf', 'Minigolf', 'Gimnasia', 'Balonmano', 'Hockey',
                     'Petanca', 'Calva', 'Artes marciales', 'Kickboxing', 'Kickball', 'Padel', 'Raquetbol',
                     'Escalada', 'Salto a la comba', 'Chito', 'Skateboard', 'Patinaje', 'Fútbol', 'Fútbol sala',
                     'Sóftbol', 'Béisbol', 'Tenis de mesa', 'Tai chi', 'Tenis', 'Tenis en pareja',
                     'Voleibol', 'Marcha rápida', 'Montañismo', 'Caminar', 'Natación', 'Nado sincronizado',
                     'Waterpolo', 'Voleibol acuático', 'Patinaje sobre hielo', 'Frontenis', 'Senderismo'])

# Generar un DataFrame sintético con datos históricos de recomendaciones de deportes
np.random.seed(42)
num_datos = 1000

# Simular datos de Fecha y Hora
# horas = [random.randint(6, 23) for _ in range(num_datos)]

# Simular datos de Clima
clima = np.random.choice(['Soleado', 'Nublado', 'Lluvioso'], num_datos)

# Simular deportes que se practican según el clima
def get_deporte_clima(clima):
    if clima == 'Soleado':
        return random.choice(['Tenis', 'Tenis de mesa'])
    elif clima == 'Lluvioso':
        return random.choice(['Dardos', 'Máquina escaladora'])
    elif clima == 'Nublado':
        return random.choice(['Senderismo', 'Salto a la comba'])

deporte_segun_clima = [get_deporte_clima(c) for c in clima]

# Crear el DataFrame
df_historico = pd.DataFrame({
    'Clima': clima,
    'DeportePracticadoSegunClima': deporte_segun_clima,
})


# Simular datos de Temperatura en el rango de 10 a 35 grados Celsius
temperatura = np.random.uniform(10, 40, num_datos)

# Simular datos de Humedad en el rango de 30 a 80 por ciento
humedad = np.random.randint(30, 81, num_datos)

# Simular datos de Edad en el rango de 18 a 65 años
edad = np.random.randint(14, 66, num_datos)

# Simular datos de peso en el rango de 50 a 100 kilogramos
peso = np.random.uniform(50, 100, num_datos)

# Simular objetivos calóricos
objetivos_caloricos = np.random.choice([0, 1, 2], num_datos)

# Simular datos de distancia en el rango de 0.5 a 20 kilómetros
distancia = np.random.uniform(0.5, 20, num_datos)
prob_hombre = 0.7
prob_mujer = 1 - prob_hombre
genero = np.random.choice(['Hombre', 'Mujer'], num_datos, p=[prob_hombre, prob_mujer])
# Simular condición física
condicion_fisica = np.random.choice([0, 1, 2], num_datos)

# Simular deportes que ha hecho antes
def get_deporte_practicado(genero):
    if genero == 'Mujer':
        return [random.choice(['Ciclismo'] * 3 + list(deportes)) for _ in range(random.randint(3, len(deportes)))]
    else:
        return [random.choice(['Calistenia'] * 3 + list(deportes)) for _ in range(random.randint(3, len(deportes)))]
    
genero = np.random.choice(['Mujer', 'Hombre'], num_datos)
deporte = [random.choice(deportes) for _ in range(1000)]
deporte_practicado = [get_deporte_practicado(g) for g in genero]

# Crear el DataFrame
df_historico = pd.DataFrame({
    # 'Hora': horas,
    'Clima': clima,
    'Temperatura (°C)': temperatura.round().astype(int),
    'Humedad': humedad,
    'Edad': edad,
    'Genero': genero,
    'Peso (Kg)': peso.round(2),
    'ObjetivoCalorico': objetivos_caloricos,
    'Distancia (Km)': distancia.round(2),
    'CondicionFisica': condicion_fisica,
    'DeportePracticado': deporte_practicado,
})

# # Convertir columna de clima en variables dummy
# clima_dummies = pd.get_dummies(df_historico['Clima'], prefix='Clima')
# df_historico = pd.concat([df_historico, clima_dummies], axis=1)

# # Convertir columna de género en variable dummy
# genero_dummies = pd.get_dummies(df_historico['Genero'], prefix='Genero')
# df_historico = pd.concat([df_historico, genero_dummies], axis=1)

# # Convertir columna de condición física en variables dummy
# condicion_fisica_dummies = pd.get_dummies(df_historico['CondicionFisica'], prefix='Condicion')
# df_historico = pd.concat([df_historico, condicion_fisica_dummies], axis=1)

# # Convertir columna de objetivo calórico en variables dummy
# objetivos_caloricos_dummies = pd.get_dummies(df_historico['ObjetivoCalorico'], prefix='Objetivo')
# df_historico = pd.concat([df_historico, objetivos_caloricos_dummies], axis=1)

# # Convertir columna de deportes practicados en variables dummy
# deporte_practicado_dummies = df_historico['DeportePracticado'].apply(lambda x: '|'.join(x)).str.get_dummies()
# df_historico = pd.concat([df_historico, deporte_practicado_dummies], axis=1)


# # Eliminar columnas originales que ya no son necesarias
df_historico.drop(['DeportePracticado'], axis=1, inplace=True)

df_historico.to_csv('../Raw_Datasets/EDA_historicos_deportes.csv', index=False)
# Mostrar el DataFrame resultante
df_historico


,Clima,Temperatura (°C),Humedad,Edad,Genero,Peso (Kg),ObjetivoCalorico,Distancia (Km),CondicionFisica
0,Lluvioso,12,59,54,Hombre,90.84,2,3.05,0
1,Soleado,26,43,52,Hombre,59.08,2,13.60,0
2,Lluvioso,22,35,26,Mujer,68.55,0,14.72,1
3,Lluvioso,39,32,24,Mujer,95.10,2,11.71,0
4,Soleado,13,45,59,Mujer,90.33,1,4.53,2
...,...,...,...,...,...,...,...,...,...
995,Nublado,38,46,63,Mujer,67.29,2,16.61,1
996,Nublado,13,53,43,Hombre,83.55,0,14.18,2
997,Lluvioso,35,50,50,Mujer,90.27,2,9.46,1
998,Lluvioso,21,80,38,Hombre,97.34,1,1.57,2


In [4]:
import pandas as pd
import numpy as np
import random

# Resto del código para generar el DataFrame df_historico...

# Ajustar la proporción de mujeres y hombres
num_mujeres = int(0.6 * num_datos)  # El 60% serán mujeres
num_hombres = num_datos - num_mujeres
genero_mujer = ['Mujer'] * num_mujeres
genero_hombre = ['Hombre'] * num_hombres
genero = genero_mujer + genero_hombre
random.shuffle(genero)

# Crear dos grupos para mujeres y hombres
df_mujeres = df_historico[df_historico['Genero_Mujer'] == 1].copy()
df_hombres = df_historico[df_historico['Genero_Hombre'] == 1].copy()

# Asignar características específicas para los grupos de mujeres
condicion_mujeres = (df_mujeres['Edad'] >= 20) & (df_mujeres['Edad'] <= 40)
df_mujeres['DeportePracticado_Correr'] = np.where(condicion_mujeres, 1, 0)
df_mujeres['DeportePracticado_Escalada'] = np.where(condicion_mujeres, 1, 0)

# Asignar características específicas para el grupo de hombres que practican pádel
condicion_hombres = (df_hombres['Edad'] >= 30) & (df_hombres['Edad'] <= 40) & (df_hombres['Peso (Kg)'] >= 60) & (df_hombres['Peso (Kg)'] <= 70)
df_hombres['DeportePracticado_Padel'] = np.where(condicion_hombres, 1, 0)

# Eliminar las columnas dummy que ya no son necesarias
df_mujeres.drop(['Genero_Hombre'], axis=1, inplace=True)
df_hombres.drop(['Genero_Mujer'], axis=1, inplace=True)

# Concatenar nuevamente los DataFrames para obtener el DataFrame desbalanceado
df_desbalanceado = pd.concat([df_mujeres, df_hombres], ignore_index=True)

# Completar valores nulos con ceros
df_desbalanceado.fillna(0, inplace=True)

# Mostrar el DataFrame desbalanceado
print(df_desbalanceado)


     Temperatura (°C)  Humedad  Edad  Peso (Kg)  Distancia (Km)  \
0                  13       45    63      80.80           13.11   
1                  36       38    30      77.95            6.49   
2                  35       44    63      75.05           17.26   
3                  18       50    62      85.31           19.21   
4                  15       64    18      76.71            4.75   
..                ...      ...   ...        ...             ...   
995                22       78    29      50.48           11.86   
996                23       71    65      92.90           13.83   
997                40       66    20      62.39            8.32   
998                38       46    53      78.31            2.93   
999                21       80    21      76.77            4.46   

     Clima_Lluvioso  Clima_Nublado  Clima_Soleado  Genero_Mujer  Condicion_0  \
0                 0              0              1           1.0            0   
1                 0              1 

In [5]:
df_desbalanceado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 75 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Temperatura (°C)            1000 non-null   int32  
 1   Humedad                     1000 non-null   int32  
 2   Edad                        1000 non-null   int32  
 3   Peso (Kg)                   1000 non-null   float64
 4   Distancia (Km)              1000 non-null   float64
 5   Clima_Lluvioso              1000 non-null   uint8  
 6   Clima_Nublado               1000 non-null   uint8  
 7   Clima_Soleado               1000 non-null   uint8  
 8   Genero_Mujer                1000 non-null   float64
 9   Condicion_0                 1000 non-null   uint8  
 10  Condicion_1                 1000 non-null   uint8  
 11  Condicion_2                 1000 non-null   uint8  
 12  Objetivo_0                  1000 non-null   uint8  
 13  Objetivo_1                  1000 n

In [1]:
import pandas as pd
import numpy as np
import random

# Lista de deportes
deportes = np.array(['BMX', 'Ciclismo', 'Ciclismo estacionario', 'Calistenia', 'Entrenamiento en circuito',
                     'Levantamiento de peso', 'Máquina escaladora', 'Aeróbicos', 'Yoga', 'Aeróbicos acuáticos',
                     'Correr', 'Atletismo', 'Bádminton', 'Baloncesto', 'Bolos', 'Billar', 'Boxeo', 'Dardos',
                     'Frisbee', 'Ultimate frisbee', 'Golf', 'Minigolf', 'Gimnasia', 'Balonmano', 'Hockey',
                     'Petanca', 'Calva', 'Artes marciales', 'Kickboxing', 'Kickball', 'Padel', 'Raquetbol',
                     'Escalada', 'Salto a la comba', 'Chito', 'Skateboard', 'Patinaje', 'Fútbol', 'Fútbol sala',
                     'Sóftbol', 'Béisbol', 'Tenis de mesa', 'Tai chi', 'Tenis', 'Tenis en pareja',
                     'Voleibol', 'Marcha rápida', 'Montañismo', 'Caminar', 'Natación', 'Nado sincronizado',
                     'Waterpolo', 'Voleibol acuático', 'Patinaje sobre hielo', 'Frontenis', 'Senderismo'])

# Generar un DataFrame sintético con datos históricos de recomendaciones de deportes
np.random.seed(42)
num_datos = 1000

# Simular datos de Clima
clima = np.random.choice(['Soleado',  'Nublado','Lluvioso'], num_datos)

# Simular datos de Temperatura en el rango de 10 a 35 grados Celsius
temperatura = np.random.uniform(10, 40, num_datos)

# Simular datos de Humedad en el rango de 30 a 80 por ciento
humedad = np.random.randint(30, 81, num_datos)

# Simular datos de Edad en el rango de 18 a 65 años
edad = np.random.randint(18, 66, num_datos)

# Simular datos de peso en el rango de 50 a 100 kilogramos
peso = np.random.uniform(50, 100, num_datos)

# Simular objetivos calóricos
objetivos_caloricos = np.random.choice([0, 1, 2], num_datos)

# Simular datos de distancia en el rango de 0.5 a 20 kilómetros
distancia = np.random.uniform(0.5, 20, num_datos)

# Simular condición física
condicion_fisica = np.random.choice([0, 1, 2], num_datos)

# Simular deportes que ha hecho antes
deporte = [random.choice(deportes) for _ in range(1000)]
deporte_practicado = [random.sample(list(deportes), random.randint(3, len(deportes))) for _ in range(num_datos)]

genero = np.random.choice(['Mujer','Hombre'], num_datos)

# Crear el DataFrame
df_historico = pd.DataFrame({
    # 'Hora': horas,
    'Clima': clima,
    'Temperatura (°C)': temperatura.round().astype(int),
    'Humedad': humedad,
    'Edad': edad,
    'Genero': genero,
    'Peso (Kg)': peso.round(2),
    'ObjetivoCalorico': objetivos_caloricos,
    'Distancia (Km)': distancia.round(2),
    'CondicionFisica': condicion_fisica,
    'DeportePracticado': deporte_practicado,
    
})

# Convertir columna de clima en variables dummy
clima_dummies = pd.get_dummies(df_historico['Clima'], prefix='Clima')
df_historico = pd.concat([df_historico, clima_dummies], axis=1)

genero_dummies = pd.get_dummies(df_historico['Genero'], prefix='Genero')
df_historico = pd.concat([df_historico, genero_dummies], axis=1)

# Convertir columna de condición física en variables dummy
condicion_fisica_dummies = pd.get_dummies(df_historico['CondicionFisica'], prefix='Condicion')
df_historico = pd.concat([df_historico, condicion_fisica_dummies], axis=1)

# Convertir columna de objetivo calórico en variables dummy
objetivos_caloricos_dummies = pd.get_dummies(df_historico['ObjetivoCalorico'], prefix='Objetivo')
df_historico = pd.concat([df_historico, objetivos_caloricos_dummies], axis=1)



# Convertir columna de deportes practicados en variables dummy
deporte_practicado_dummies = df_historico['DeportePracticado'].apply(lambda x: '|'.join(x)).str.get_dummies()
df_historico = pd.concat([df_historico, deporte_practicado_dummies], axis=1)

# Eliminar columnas originales que ya no son necesarias
df_historico.drop(['Clima', 'CondicionFisica', 'ObjetivoCalorico','Genero','DeportePracticado'], axis=1, inplace=True)




# Ajustar la proporción de mujeres y hombres
num_mujeres = int(0.6 * num_datos)  # El 60% serán mujeres
num_hombres = num_datos - num_mujeres
genero_mujer = ['Mujer'] * num_mujeres
genero_hombre = ['Hombre'] * num_hombres
genero = genero_mujer + genero_hombre
random.shuffle(genero)

# Crear dos grupos para mujeres y hombres
df_mujeres = df_historico[df_historico['Genero_Mujer'] == 1].copy()
df_hombres = df_historico[df_historico['Genero_Hombre'] == 1].copy()

# Asignar características específicas para los grupos de mujeres
df_mujeres.loc[(df_mujeres['Edad'] >= 20) & (df_mujeres['Edad'] <= 40), 'DeportePracticado_Correr'] = 1
df_mujeres.loc[(df_mujeres['Edad'] >= 20) & (df_mujeres['Edad'] <= 40), 'DeportePracticado_Escalada'] = 1

# Asignar características específicas para el grupo de hombres que practican pádel
df_hombres.loc[
    (df_hombres['Edad'] >= 30) & (df_hombres['Edad'] <= 40) &
    (df_hombres['Peso (Kg)'] >= 60) & (df_hombres['Peso (Kg)'] <= 70),
    'DeportePracticado_Padel'
] = 1

# Eliminar las columnas dummy que ya no son necesarias
df_mujeres.drop(['Genero_Hombre'], axis=1, inplace=True)
df_hombres.drop(['Genero_Mujer'], axis=1, inplace=True)

# Concatenar nuevamente los DataFrames para obtener el DataFrame desbalanceado
df_desbalanceado = pd.concat([df_mujeres, df_hombres], ignore_index=True)

# Mostrar el DataFrame desbalanceado
df_desbalanceado


,Temperatura (°C),Humedad,Edad,Peso (Kg),Distancia (Km),Clima_Lluvioso,Clima_Nublado,Clima_Soleado,Genero_Mujer,Condicion_0,...,Tenis en pareja,Ultimate frisbee,Voleibol,Voleibol acuático,Waterpolo,Yoga,DeportePracticado_Correr,DeportePracticado_Escalada,Genero_Hombre,DeportePracticado_Padel
0,13,45,63,80.80,13.11,0,0,1,1.0,0,...,1,0,1,1,1,1,NaN,NaN,NaN,NaN
1,36,38,30,77.95,6.49,0,1,0,1.0,1,...,1,1,1,1,0,1,1.0,1.0,NaN,NaN
2,35,44,63,75.05,17.26,1,0,0,1.0,0,...,0,0,1,1,0,1,NaN,NaN,NaN,NaN
3,18,50,62,85.31,19.21,1,0,0,1.0,1,...,1,1,1,1,1,0,NaN,NaN,NaN,NaN
4,15,64,18,76.71,4.75,1,0,0,1.0,0,...,1,0,1,0,1,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,22,78,29,50.48,11.86,0,0,1,NaN,1,...,1,0,1,0,1,1,NaN,NaN,1.0,NaN
996,23,71,65,92.90,13.83,0,1,0,NaN,0,...,0,1,0,0,0,0,NaN,NaN,1.0,NaN
997,40,66,20,62.39,8.32,0,1,0,NaN,1,...,0,1,1,1,0,1,NaN,NaN,1.0,NaN
998,38,46,53,78.31,2.93,0,1,0,NaN,1,...,1,1,1,1,1,1,NaN,NaN,1.0,NaN


In [16]:
df_historico = pd.read_csv('../Raw_Datasets/datos_historicos_deportes.csv',sep=',') 
df_historico.columns

Index(['Temperatura (°C)', 'Humedad', 'Edad', 'Peso (Kg)', 'Distancia (Km)',
       'Deporte', 'Clima_Lluvioso', 'Clima_Nublado', 'Clima_Soleado',
       'Genero_Hombre', 'Genero_Mujer', 'Condicion_0', 'Condicion_1',
       'Condicion_2', 'Objetivo_0', 'Objetivo_1', 'Objetivo_2', 'Aeróbicos',
       'Aeróbicos acuáticos', 'Artes marciales', 'Atletismo', 'BMX',
       'Baloncesto', 'Balonmano', 'Billar', 'Bolos', 'Boxeo', 'Bádminton',
       'Béisbol', 'Calistenia', 'Calva', 'Caminar', 'Chito', 'Ciclismo',
       'Ciclismo estacionario', 'Correr', 'Dardos',
       'Entrenamiento en circuito', 'Escalada', 'Frisbee', 'Frontenis',
       'Fútbol', 'Fútbol sala', 'Gimnasia', 'Golf', 'Hockey', 'Kickball',
       'Kickboxing', 'Levantamiento de peso', 'Marcha rápida', 'Minigolf',
       'Montañismo', 'Máquina escaladora', 'Nado sincronizado', 'Natación',
       'Padel', 'Patinaje', 'Patinaje sobre hielo', 'Petanca', 'Raquetbol',
       'Salto a la comba', 'Senderismo', 'Skateboard', 'Sóftbol'

In [10]:
df_historico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 73 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Temperatura (°C)           1000 non-null   int32  
 1   Humedad                    1000 non-null   int32  
 2   Edad                       1000 non-null   int32  
 3   Peso (Kg)                  1000 non-null   float64
 4   Distancia (Km)             1000 non-null   float64
 5   Deporte                    1000 non-null   int32  
 6   Clima_Lluvioso             1000 non-null   uint8  
 7   Clima_Nublado              1000 non-null   uint8  
 8   Clima_Soleado              1000 non-null   uint8  
 9   Genero_Hombre              1000 non-null   uint8  
 10  Genero_Mujer               1000 non-null   uint8  
 11  Condicion_0                1000 non-null   uint8  
 12  Condicion_1                1000 non-null   uint8  
 13  Condicion_2                1000 non-null   uint8 



# Desafío de Tripulaciones · Grupo 5 · Data Science

## Objetivo de Data en la Aplicación

Este proyecto tiene como objetivo principal optimizar las actividades deportivas durante climas adversos, como olas de calor, mediante la implementación de un sistema de recomendación basado en datos. A continuación, se destacan los puntos clave del proceso de data en la aplicación:

1. **Recolección de Datos:** Se han utilizado diversas fuentes de datos para obtener información detallada sobre ubicaciones y actividades deportivas en el área de Madrid.

3. **Diseño de Base de Datos (BBDD):** Se diseñó una base de datos con los compañeros de full stack,  completa para almacenar los datos procesados y permitir su fácil acceso. Para llevar a cabo esta tarea, se utilizaron herramientas como Excalidraw

4. **Tratamiento de Datos:** Se aplicaron técnicas de procesamiento de datos para obtener información relevante y prepararla para su análisis.  

5. **Generación de Dataset de Usuarios:** Se crearon datasets que contienen perfiles de usuarios, los cuales son fundamentales para el entrenamiento de modelos y la generación de recomendaciones personalizadas.

6. **Análisis Exploratorio de Datos (EDA):** Se llevó a cabo un análisis exploratorio de los datos para comprender mejor las preferencias y patrones de los usuarios.

7. **Diseño y Entrenamiento de Modelos:** Se diseñaron y entrenaron modelos de recomendación basados en las preferencias y comportamientos identificados en los datos.

8. **Recomendador:** se a creado una matriz de similitud basada en las características de diferentes deportes para crear un recomendador basado en preferencias deportivas, cuyos resultados se relacionan con los datos obtenidos del modelo predictivo para así recomendar ubicaciones en el área de Madrid que cumplan con los requisitos de preferencias, predicción y cercanía del usuario a los mismos

9. **Web Scraping de Actividades:** Se automatizó el proceso de recolección de datos diarios sobre actividades ofrecidas por la comunidad en centros climatizados.

10. **Automatización de Actualización de Tabla de Actividades:** Se desarrolló un sistema automatizado para actualizar la tabla de actividades en la base de datos con los datos recopilados mediante web scraping, el objetivo consiste en poder recomendar actividades diarias ofrecidas por la comunidad en centros climatizados, independientemente de las diferentes características y preferencias de los usuarios.

11. **Generación de API:** Se creó una API que permite el consumo de los datos y las recomendaciones desde la aplicación.

1. **Recursos utilizadas:**

     [herramientas](../herramientas.txt)

     [Fuentes](../fuentes.md)

## Equipo

Este proyecto ha sido posible gracias a las valiosas contribuciones de cada uno de los siguientes miembros, cuyos esfuerzos combinados han hecho realidad su desarrollo:

- [Erika Guzmán](https://github.com/Erikahenriquez78)
- [Giacomo Salerno](https://github.com/GiamoSalerno)
- [Jaime Puchol](https://github.com/jpucholv)

¡Agradecemos a todo el equipo por su dedicación y colaboración en la creación de esta aplicación innovadora!

